In [1]:
# Standard imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
# import mavenn
import sys

# Load mavenn
import mavenn

In [2]:
model = mavenn.load('gb1_ge_additive')

2021-12-31 10:59:58.104975: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model loaded from these files:
	gb1_ge_additive.pickle
	gb1_ge_additive.h5


In [3]:
# Load example data (WHICH CONTAINS DOUBLE MUTANTS ONLY!)
data_df = mavenn.load_example_dataset('gb1')

# Separate test from data_df
ix_test = data_df['set']=='test'
test_df = data_df[ix_test].reset_index(drop=True)
print(f'test N: {len(test_df):,}')

# Remove test data from data_df
data_df = data_df[~ix_test].reset_index(drop=True)
print(f'training + validation N: {len(data_df):,}')
data_df.head(10)

test N: 26,364
training + validation N: 504,373


,set,dist,input_ct,selected_ct,y,x
0,training,2,173,33,-3.145154,AAKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDD...
1,training,2,18,8,-1.867676,ACKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDD...
2,training,2,66,2,-5.270800,ADKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDD...
3,training,2,72,1,-5.979498,AEKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDD...
4,training,2,69,168,0.481923,AFKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDD...
5,training,2,108,1,-6.557858,AGKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDD...
6,training,2,48,93,0.150206,AHKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDD...
7,training,2,119,49,-2.052708,AIKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDD...
8,training,2,252,79,-2.450739,ALKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDD...
9,training,2,110,116,-0.713724,AMKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDD...


In [4]:
from numpy.random import default_rng
N = 300000

rng = default_rng()
# size here represents the number of double mutants
# uniformly randomly pick some number of double mutants

numbers = rng.choice(len(data_df), size=N, replace=False) 

In [5]:
data_df_N = data_df.loc[numbers].reset_index(drop=True).copy()

In [6]:
# simulate data from loaded model
sim_df = model.simulate_dataset(template_df=data_df_N)

In [7]:
sim_df

,set,phi,yhat,y,x
0,training,0.634462,-1.117043,-1.143524,QYKLILNGKTLKSETTTEAVDAATAEKVFKQYANDNGVDGEWTWDD...
1,training,-0.585621,-6.513978,-6.392124,QYKIILNGKTLKGETTTEAVDAATAEKVFKQYAGDNGVDGEWTYDD...
2,training,-2.285171,-7.901658,-7.598669,QYKLILNGKTLKGETTTEAVDAATASKVFKQYANDNGVDGEWTYDD...
3,training,-1.104050,-7.409938,-5.700406,QYKLILNGKTLKGETTTEAVDAATAEKVFKQYAPDNGVCGEWTYDD...
4,validation,-0.998027,-7.286275,-7.357221,QYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDW...
...,...,...,...,...,...
299995,training,0.338397,-2.599643,-2.499728,QYKLIYNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYHD...
299996,training,-0.154529,-5.028903,-3.956814,QYKLILNGKTLKGHTTTEAVDAATAEKVFKQYAFDNGVDGEWTYDD...
299997,training,-1.395266,-7.646314,-7.720807,QYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGENVYDD...
299998,training,-0.732591,-6.851542,-4.389921,QYKLILNGKTLKGETFTEAVDAATAEKVFKQYANDNGVDWEWTYDD...


In [8]:
# load data to compare ground-truth model
x_test = test_df['x'].values
y_test = test_df['y'].values
phi_test = model.x_to_phi(x_test)
yhat_test = model.x_to_yhat(x_test)
phi_lim = [min(phi_test)-.5, max(phi_test)+.5]
phi_grid = np.linspace(phi_lim[0], phi_lim[1], 1000)
yhat_grid = model.phi_to_yhat(phi_grid)

In [9]:
# get pairwise theta from model trained on experimental data
theta_lc = model.get_theta()['theta_lc']
# drop nans to plotting
theta_lc = theta_lc[~np.isnan(theta_lc)]

In [10]:
# # Split simulated data into trianing and test sets as
# # with the model trained on experimental data
# ix = sim_df['training_set']
# sim_train_df = sim_df[ix]
# print(f'simulated training N: {len(sim_train_df):,}')
# sim_test_df = sim_df[~ix]
# print(f'simulated testing N: {len(sim_test_df):,}')

sim_train_df = sim_df.copy()

In [11]:
# define sequence length
L = len(sim_df['x'][0])

## Train models on simulated data

In [12]:
models = []
training_times = []

for model_index in range(10):
    
    print(f'training model {model_index}')
    sim_model = mavenn.Model(L=L,
                         alphabet='protein',
                         gpmap_type='additive', 
                         regression_type='GE',
                         ge_noise_model_type='Gaussian',
                         ge_heteroskedasticity_order=2)

    # Set simulated training data
    sim_model.set_data(x=sim_train_df['x'],
                   y=sim_train_df['y'],
                   shuffle=True,
                   verbose=False)
    
    start_time = time.time()
    # Fit model to data
    sim_model.fit(learning_rate=.005,
              epochs=1000,
              try_tqdm=False,    
              batch_size=200,
              early_stopping=True,
              early_stopping_patience=30,
              linear_initialization=True,
              verbose=False)
    
    training_time = time.time() - start_time
    
    training_times.append(training_time)
    models.append(sim_model)

training model 0
Training time: 93.9 seconds
training model 1
Training time: 127.1 seconds
training model 2
Training time: 121.5 seconds
training model 3
Training time: 100.0 seconds
training model 4
Training time: 118.0 seconds
training model 5
Training time: 145.5 seconds
training model 6
Training time: 118.6 seconds
training model 7
Training time: 93.5 seconds
training model 8
Training time: 105.6 seconds
training model 9
Training time: 110.5 seconds


In [13]:
# make directory to save results
directory = 'N_'+str(len(sim_df))
!mkdir models/$directory

mkdir: models/N_300000: File exists


In [14]:
model_Rsqs = []
for model_index in range(len(models)):
    print(f' Model: {model_index}, $R^2$: {np.corrcoef(models[model_index].x_to_yhat(x_test),yhat_test)[0][1]**2}')
    model_Rsqs.append(np.corrcoef(models[model_index].x_to_yhat(x_test),yhat_test)[0][1]**2)
    models[model_index].save(f'models/{directory}/model_{model_index}')

 Model: 0, $R^2$: 0.9985265202338827
Model saved to these files:
	models/N_300000/model_0.pickle
	models/N_300000/model_0.h5
 Model: 1, $R^2$: 0.9984153675890678
Model saved to these files:
	models/N_300000/model_1.pickle
	models/N_300000/model_1.h5
 Model: 2, $R^2$: 0.9984569396046553
Model saved to these files:
	models/N_300000/model_2.pickle
	models/N_300000/model_2.h5
 Model: 3, $R^2$: 0.9985792426895276
Model saved to these files:
	models/N_300000/model_3.pickle
	models/N_300000/model_3.h5
 Model: 4, $R^2$: 0.9987774445865912
Model saved to these files:
	models/N_300000/model_4.pickle
	models/N_300000/model_4.h5
 Model: 5, $R^2$: 0.9986649575871799
Model saved to these files:
	models/N_300000/model_5.pickle
	models/N_300000/model_5.h5
 Model: 6, $R^2$: 0.997673754997059
Model saved to these files:
	models/N_300000/model_6.pickle
	models/N_300000/model_6.h5
 Model: 7, $R^2$: 0.9986833821203004
Model saved to these files:
	models/N_300000/model_7.pickle
	models/N_300000/model_7.h5
 

In [15]:
#np.savetxt('models/N_24K/training_times.txt',training_times)
#np.savetxt('models/N_24K/model_Rsqs.txt',model_Rsqs)

np.savetxt(f'models/{directory}/training_times.txt',training_times)
np.savetxt(f'models/{directory}/model_Rsqs.txt',model_Rsqs)

In [16]:
training_times

[93.98594689369202,
 127.1999089717865,
 121.60798525810242,
 100.10070300102234,
 118.09328508377075,
 145.5615038871765,
 118.64710664749146,
 93.53135991096497,
 105.6686840057373,
 110.60171222686768]